# Concise Implementation of Linear Regression

In [1]:
import d2l
from mxnet import autograd, np, npx, gluon
npx.set_np()

true_w = np.array([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

Reading Data

In [2]:
def load_array(data_arrays, batch_size, is_train=True):
    dataset = gluon.data.ArrayDataset(*data_arrays)
    return gluon.data.DataLoader(dataset, batch_size, shuffle=is_train)
    
batch_size = 10
data_iter = load_array((features, labels), batch_size)
for X, y in data_iter:
    print('X = \n', X, '\ny = \n', y)
    break

X = 
 [[-0.3300724   0.09964364]
 [-1.4632133  -0.43755293]
 [ 0.83400416 -2.3646805 ]
 [ 0.9561315  -0.8629183 ]
 [-1.02716     0.07130642]
 [ 1.6263341   0.49180838]
 [ 1.3684256  -0.74242383]
 [-0.12898901  0.9547114 ]
 [ 1.2054669  -0.99645287]
 [-0.42176926 -0.6764356 ]] 
y = 
 [ 3.196445   2.7593803 13.898395   9.049869   1.9153634  5.772378
  9.457492   0.6997888  9.992633   5.6543956]


Define the Model and initialize Model Parameters

In [3]:
from mxnet.gluon import nn
from mxnet import init

net = nn.Sequential()
net.add(nn.Dense(1))
net.initialize(init.Normal(sigma=0.01))

Define the loss function and optimization algorithm

In [4]:
from mxnet import gluon

loss = gluon.loss.L2Loss() 
trainer = gluon.Trainer(net.collect_params(),
                        'sgd', {'learning_rate': 0.03})

Training

In [5]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch, l.mean()))
    
w = net[0].weight.data()
print('Error in estimating w', true_w.reshape(w.shape) - w)
b = net[0].bias.data()
print('Error in estimating b', true_b - b)    

epoch 1, loss: 0.040392
epoch 2, loss: 0.000152
epoch 3, loss: 0.000051
Error in estimating w [[ 0.00034511 -0.0005765 ]]
Error in estimating b [0.00081873]
